Based on:
- https://www.kaggle.com/ashishpatel26/preprocessing-lightgbm-xgboost
- https://github.com/Shitao-zz/Kaggle-House-Prices-Advanced-Regression-Techniques
- https://www.kaggle.com/vinicius150987/regression-boston-sklearn-dataset

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
#import swifter

from category_encoders import HelmertEncoder, SumEncoder, OneHotEncoder
#from fancyimpute import KNN, SoftImpute, BiScaler, IterativeSVD, MatrixFactorization, SimpleFill

from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, RobustScaler
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA, TruncatedSVD, FastICA
from sklearn.model_selection import KFold
from sklearn.random_projection import GaussianRandomProjection, SparseRandomProjection
from sklearn.preprocessing import normalize
from sklearn.manifold import TSNE
from sklearn.datasets import load_boston, make_classification, make_regression
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer
from sklearn.linear_model import BayesianRidge, SGDRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error, mean_squared_log_error, mean_absolute_error, median_absolute_error

pd.options.mode.chained_assignment = None 
pd.options.display.max_columns = None

In [ ]:
cat_cols = ['TODO']
num_cols = ['TODO']
target = ['label']
len(cat_cols) + len(num_cols)

In [ ]:
encode_cat = False
scaling = True

save_data = False
#drop_nan_rows = False
#drop_error_data = False

random_state = 42

#imputer_num = IterativeImputer(estimator=SGDRegressor(loss='huber', penalty='l1'), verbose=2, max_iter=1)
#imputer_num = IterativeImputer(estimator=BayesianRidge(), verbose=2, max_iter=30)
#imputer_num = IterativeImputer(estimator=ExtraTreesRegressor(n_estimators=10, criterion="mae", n_jobs=-1, random_state=random_state), verbose=2, max_iter=5, n_nearest_features=3)
#imputer_num = IterativeImputer(estimator=KNeighborsRegressor(n_jobs=-1), n_nearest_features=5, verbose=2, max_iter=5)
imputer_num = SimpleImputer()
#imputer_num = KNNImputer(n_neighbors=5) # also slow https://github.com/scikit-learn/scikit-learn/issues/18186, 33min for 200k rows TODO: normalize before! but gives error :(

imputer_cat = SimpleImputer(strategy='most_frequent')

scaler = StandardScaler()
encoder = OneHotEncoder(cat_cols)

In [ ]:
def save_memory(df, compress_float=True):
    features = df.columns
    for i in range( df.shape[1] ):
        if df.dtypes[i] == 'uint8':
            df[features[i]] = df[features[i]].astype( np.int8 )
        elif df.dtypes[i] == 'bool':
            df[features[i]] = df[features[i]].astype( np.int8 )
        elif df.dtypes[i] == 'uint32':
            df[features[i]] = df[features[i]].astype( np.int32 )
        elif df.dtypes[i] == 'int64':
            df[features[i]] = df[features[i]].astype( np.int32 )
        elif df.dtypes[i] == 'float64' and compress_float:
            df[features[i]] = df[features[i]].astype( np.float32 )

In [ ]:
df = pd.read_csv('data/data_original.csv')
df.head(5)

In [ ]:
# remove inf values
df = df.replace([np.inf, -np.inf], np.nan)

for cat in cat_cols:
    df[cat] = df[cat].astype("category")

# replace string nan 
df = df.replace('nan', np.nan)

df = df[target + cat_cols + num_cols]

print(df.shape)
print(df.dtypes)
df.head()

In [ ]:
df_test = pd.read_csv('data/data_test.csv')

# remove inf values
df_test = df_test.replace([np.inf, -np.inf], np.nan)

for cat in cat_cols:
    df_test[cat] = df_test[cat].astype("category")

# replace string nan 
df_test = df_test.replace('nan', np.nan)

X_test = df_test[['id'] + cat_cols + num_cols]

print(X_test.shape)
print(X_test.dtypes)
X_test.head()

In [ ]:
X = df.drop(target, axis=1)
y = df[target]

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1, random_state=random_state)
print(X_train.shape, y_train.shape, X_valid.shape, y_valid.shape)
X_train.head()

In [ ]:
# Imputation
X_train[num_cols] = imputer_num.fit_transform(X_train[num_cols])
X_valid[num_cols] = imputer_num.transform(X_valid[num_cols])
X_test[num_cols] = imputer_num.transform(X_test[num_cols])

X_train[cat_cols] = imputer_cat.fit_transform(X_train[cat_cols])
X_valid[cat_cols] = imputer_cat.transform(X_valid[cat_cols])
X_test[cat_cols] = imputer_cat.transform(X_test[cat_cols])

X_train.head()

In [ ]:
# NORMALIZE/STANDARDIZE
if scaling:
    X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
    X_valid[num_cols] = scaler.transform(X_valid[num_cols])
    X_test[num_cols] = scaler.transform(X_test[num_cols])

X_train.head()

In [ ]:
# One Hot Encoding
if encode_cat:
    X_train = encoder.fit_transform(X_train, y_train)
    X_valid = encoder.transform(X_valid)
    X_test = encoder.transform(X_test)
else:
    for cat in cat_cols:
        X_train[cat] = X_train[cat].astype("category")
        X_valid[cat] = X_valid[cat].astype("category")
        X_test[cat] = X_test[cat].astype("category")
        print(f"{cat} {X_test[cat].dtype}")
y_train = y_train.astype("float32")
y_valid = y_valid.astype("float32")
X_train.head()

In [ ]:
X_test.head(10)

In [ ]:
from lightgbm import LGBMRegressor

model = LGBMRegressor(n_jobs=-1, objective="regression_l1",
            boosting_type='dart',
            learning_rate=0.13110563910426518,
            max_depth=50,
            n_estimators=1000,
            num_leaves=200,
            reg_alpha=1.5921278701590583e-05,
            reg_lambda=1.0,
            subsample_for_bin=200000
)
model.fit(X_train, y_train)

print("Train MAE: ", mean_absolute_error(y_train, np.abs(model.predict(X_train))))
print("MeanAE: ", mean_absolute_error(y_valid, np.abs(model.predict(X_valid))))
print("MedianAE: ", median_absolute_error(y_valid, np.abs(model.predict(X_valid))))

In [ ]:
#raise SystemExit("Stop right there!")
np.abs(model.predict(X_test.drop('DW_PET_VST_ID', axis=1)))

In [ ]:
save_memory(X_train, compress_float=False)
save_memory(X_test, compress_float=False)
print(X_train.dtypes)

if encode_cat:
    file_name = "_pre_simple.csv"
else:
    file_name = "_pre_simple_no_enc.csv"

if save_data:
    X_train.to_csv('data/X_train' + file_name, index=False)
    X_valid.to_csv('data/X_valid' + file_name, index=False)
    X_test.to_csv('data/X_test' + file_name, index=False)
    y_train.to_csv('data/y_train' + file_name, index=False)
    y_valid.to_csv('data/y_valid' + file_name, index=False)

In [ ]:
flist = num_cols
X_train = X_train.reset_index(drop=True)
X_valid = X_valid.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)

# perform a polynomial features transform of the dataset
trans = PolynomialFeatures(degree=2) # exponential! only 2 or 3
trans_train = trans.fit_transform(X_train[flist])
trans_valid = trans.transform(X_valid[flist])
trans_test = trans.transform(X_test[flist])

X_train = pd.concat([X_train, pd.DataFrame(trans_train, columns=trans.get_feature_names(X_train[flist].columns))], axis=1)
X_valid = pd.concat([X_valid, pd.DataFrame(trans_valid, columns=trans.get_feature_names(X_valid[flist].columns))], axis=1)
X_test = pd.concat([X_test, pd.DataFrame(trans_test, columns=trans.get_feature_names(X_test[flist].columns))], axis=1)

print(X_train.shape, y_train.shape)
X_train.head()

In [ ]:
def kmeans(X_tr, X_vl, X_te, flist):
    flist_kmeans = []
    for ncl in range(2,11):
        cls = KMeans(n_clusters=ncl)
        cls.fit_predict(X_train[flist].values)
        X_tr['kmeans_cluster_'+str(ncl)] = cls.predict(X_tr[flist].values)
        X_vl['kmeans_cluster_'+str(ncl)] = cls.predict(X_vl[flist].values)
        X_te['kmeans_cluster_'+str(ncl)] = cls.predict(X_te[flist].values)
        flist_kmeans.append('kmeans_cluster_'+str(ncl))
    print(flist_kmeans)
    
    return X_tr, X_vl, X_te

In [ ]:
flist = num_cols
X_train, X_valid, X_test = kmeans(X_train, X_valid, X_test, flist)
X_train.head()

In [ ]:
N_COMP = 20           ### Number of decomposition components ###
flist = num_cols

print("\nStart decomposition process...")
print("PCA")
pca = PCA(n_components=N_COMP, random_state=random_state)
pca_results_train = pca.fit_transform(X_train[flist].values)
pca_results_valid = pca.transform(X_valid[flist].values)
pca_results_test = pca.transform(X_test[flist].values)
print(pca.explained_variance_ratio_)

print("tSVD")
tsvd = TruncatedSVD(n_components=N_COMP, random_state=random_state)
tsvd_results_train = tsvd.fit_transform(X_train[flist].values)
tsvd_results_valid = tsvd.transform(X_valid[flist].values)
tsvd_results_test = tsvd.transform(X_test[flist].values)

print("ICA")
ica = FastICA(n_components=N_COMP, random_state=random_state)
ica_results_train = ica.fit_transform(X_train[flist].values)
ica_results_valid = ica.transform(X_valid[flist].values)
ica_results_test = ica.transform(X_test[flist].values)

print("GRP")
grp = GaussianRandomProjection(n_components=N_COMP, eps=0.1, random_state=random_state)
grp_results_train = grp.fit_transform(X_train[flist].values)
grp_results_valid = grp.transform(X_valid[flist].values)
grp_results_test = grp.transform(X_test[flist].values)

print("SRP")
srp = SparseRandomProjection(n_components=N_COMP, dense_output=True, random_state=random_state)
srp_results_train = srp.fit_transform(X_train[flist].values)
srp_results_valid = srp.transform(X_valid[flist].values)
srp_results_test = srp.transform(X_test[flist].values)

print("Append decomposition components to datasets...")
for i in range(1, N_COMP + 1):
    X_train['pca_' + str(i)] = pca_results_train[:, i - 1]
    X_valid['pca_' + str(i)] = pca_results_valid[:, i - 1]
    X_test['pca_' + str(i)] = pca_results_test[:, i - 1]
    
    X_train['ica_' + str(i)] = ica_results_train[:, i - 1]
    X_valid['ica_' + str(i)] = ica_results_valid[:, i - 1]
    X_test['ica_' + str(i)] = ica_results_test[:, i - 1]

    X_train['tsvd_' + str(i)] = tsvd_results_train[:, i - 1]
    X_valid['tsvd_' + str(i)] = tsvd_results_valid[:, i - 1]
    X_test['tsvd_' + str(i)] = tsvd_results_test[:, i - 1]

    X_train['grp_' + str(i)] = grp_results_train[:, i - 1]
    X_valid['grp_' + str(i)] = grp_results_valid[:, i - 1]
    X_test['grp_' + str(i)] = grp_results_test[:, i - 1]

    X_train['srp_' + str(i)] = srp_results_train[:, i - 1]
    X_valid['srp_' + str(i)] = srp_results_valid[:, i - 1]
    X_test['srp_' + str(i)] = srp_results_test[:, i - 1]
    
print('\nTrain shape: {}\nTest shape: {}'.format(X_train.shape, X_test.shape))
X_train.head()

In [ ]:
from lightgbm import LGBMRegressor

model = LGBMRegressor(n_jobs=-1, objective="regression_l1",
            boosting_type='dart',
            learning_rate=0.13110563910426518,
            max_depth=50,
            n_estimators=1000,
            num_leaves=200,
            reg_alpha=1.5921278701590583e-05,
            reg_lambda=1.0,
            subsample_for_bin=200000
)

model.fit(X_train, y_train)

print("Train MAE: ", mean_absolute_error(y_train, model.predict(X_train).abs()))
print("MeanAE: ", mean_absolute_error(y_valid, model.predict(X_valid).abs()))
print("MedianAE: ", median_absolute_error(y_valid, model.predict(X_valid).abs()))

In [ ]:
sub_df = pd.DataFrame(X_test['id'])
sub_df['AgeEstimate'] = model.predict(X_test.drop('id', axis=1)).abs()
sub_df.head()

In [ ]:
sub_df.to_csv("data/submission.csv", index=False)
sub_df.shape

In [ ]:
save_memory(X_train)
save_memory(X_test)

if save_data:
    X_train.to_csv('data/X_train.csv', index=False)
    X_valid.to_csv('data/X_valid.csv', index=False)
    X_test.to_csv('data/X_test.csv', index=False)
    y_train.to_csv('data/y_train.csv', index=False)
    y_valid.to_csv('data/y_valid.csv', index=False)